<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_10_Memory_Systems_Multi_Layer_Information_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Tutorial 9, you built circuit networks where agents collaborate on tasks.

 Now we're adding sophisticated memory systems - the ability to store retrieve, and reason with different types of information across time and context.

What you'll build:

• Multi-layered memory architecture (working, episodic, semantic, procedural, emotional)

• Memory consolidation and decay mechanisms

• Cross-memory querying and association systems

• Memory-aware agents with context retention

• Knowledge graph integration for semantic memory

• Memory visualization and debugging tools

Why this matters:

Real intelligence requires memory - not just storing facts, but understanding
context, learning from experience, and building knowledge over time. Memory
systems enable agents to become truly intelligent by accumulating wisdom
and adapting behavior based on past experiences.

By the end, you'll understand:

• How different memory types serve different cognitive functions

• Memory consolidation from working to long-term storage

• Cross-memory association and retrieval strategies

• Memory-driven decision making and learning

• Production memory system architecture and optimization



In [1]:
print("Tutorial 10: Memory Systems - Multi-Layer Information Storage")
print("=" * 60)
print()
print("Building sophisticated memory architectures for intelligent agents...")
print()

Tutorial 10: Memory Systems - Multi-Layer Information Storage

Building sophisticated memory architectures for intelligent agents...



In [2]:
# Essential imports
import uuid
import time
import threading
import queue
import json
import pickle
import hashlib
import math
import heapq
from abc import ABC, abstractmethod
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Optional, Set, Callable, Union, Tuple, Iterator
from enum import Enum
from collections import defaultdict, deque, OrderedDict, namedtuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import sqlite3
import weakref
from pathlib import Path


In [3]:
# Import our foundation from previous tutorials
AgentID = str
MessageID = str
CircuitID = str
MemoryID = str

class MemoryType(Enum):
    """Different types of memory systems"""
    WORKING = "working"         # Short-term active information
    EPISODIC = "episodic"      # Personal experiences and events
    SEMANTIC = "semantic"      # Facts and knowledge
    PROCEDURAL = "procedural"  # Skills and procedures
    EMOTIONAL = "emotional"    # Emotional associations and responses

class MemoryPriority(Enum):
    """Memory importance levels"""
    CRITICAL = 5    # Never forget
    HIGH = 4        # Important information
    NORMAL = 3      # Standard information
    LOW = 2         # Background information
    TRANSIENT = 1   # Temporary information

class ConsolidationStatus(Enum):
    """Memory consolidation states"""
    FRESH = "fresh"                 # Just created
    REHEARSED = "rehearsed"        # Recently accessed
    CONSOLIDATING = "consolidating" # Being moved to long-term
    STABLE = "stable"              # Firmly established
    DECAYING = "decaying"          # Losing strength
    ARCHIVED = "archived"          # Moved to cold storage

@dataclass
class MemoryItem:
    """
    A single memory item with metadata and content

    This represents any piece of information stored in memory,
    from facts to experiences to learned procedures.
    """
    id: MemoryID
    content: Any                                    # The actual memory content
    memory_type: MemoryType                        # Which memory system
    priority: MemoryPriority = MemoryPriority.NORMAL
    created_at: float = field(default_factory=time.time)
    last_accessed: float = field(default_factory=time.time)
    access_count: int = 0                          # How often accessed
    strength: float = 1.0                         # Memory strength (0-1)
    consolidation_status: ConsolidationStatus = ConsolidationStatus.FRESH

    # Contextual information
    context: Dict[str, Any] = field(default_factory=dict)
    tags: Set[str] = field(default_factory=set)
    associations: Set[MemoryID] = field(default_factory=set)

    # Emotional and importance weighting
    emotional_valence: float = 0.0                 # -1 (negative) to +1 (positive)
    importance_score: float = 0.5                  # 0-1 importance rating

    # Source and provenance
    source: str = "unknown"                        # Where this memory came from
    confidence: float = 1.0                        # Confidence in memory accuracy

    def access(self):
        """Record that this memory was accessed"""
        self.last_accessed = time.time()
        self.access_count += 1

        # Boost strength on access (spaced repetition effect)
        strength_boost = 0.1 / (1 + math.log(self.access_count))
        self.strength = min(1.0, self.strength + strength_boost)

    def decay(self, decay_rate: float = 0.01):
        """Apply memory decay over time"""
        time_factor = (time.time() - self.last_accessed) / (24 * 3600)  # Days
        decay_amount = decay_rate * time_factor * (1 - self.importance_score)
        self.strength = max(0.0, self.strength - decay_amount)

        # Update consolidation status based on strength
        if self.strength < 0.1:
            self.consolidation_status = ConsolidationStatus.DECAYING
        elif self.strength > 0.8 and self.access_count > 5:
            self.consolidation_status = ConsolidationStatus.STABLE

    def add_association(self, memory_id: MemoryID):
        """Add an association to another memory"""
        self.associations.add(memory_id)

    def get_age(self) -> float:
        """Get memory age in days"""
        return (time.time() - self.created_at) / (24 * 3600)

    def get_recency(self) -> float:
        """Get recency of last access in days"""
        return (time.time() - self.last_accessed) / (24 * 3600)

    def calculate_retrieval_score(self, query_context: Dict[str, Any] = None) -> float:
        """Calculate how relevant this memory is for retrieval"""
        # Base score from strength and recency
        recency_factor = 1.0 / (1.0 + self.get_recency())
        base_score = self.strength * recency_factor * self.importance_score

        # Context matching bonus
        context_bonus = 0.0
        if query_context and self.context:
            matching_keys = set(query_context.keys()) & set(self.context.keys())
            if matching_keys:
                context_bonus = len(matching_keys) / len(self.context)

        # Priority bonus
        priority_bonus = self.priority.value / 5.0

        return base_score + context_bonus * 0.3 + priority_bonus * 0.2

class MemoryQuery:
    """
    A query for retrieving memories

    This defines what we're looking for in memory and how
    to rank and filter the results.
    """

    def __init__(self,
                 content_query: str = "",
                 memory_types: List[MemoryType] = None,
                 context_filter: Dict[str, Any] = None,
                 tags_filter: Set[str] = None,
                 min_strength: float = 0.1,
                 max_age_days: float = None,
                 limit: int = 10):
        self.content_query = content_query
        self.memory_types = memory_types or list(MemoryType)
        self.context_filter = context_filter or {}
        self.tags_filter = tags_filter or set()
        self.min_strength = min_strength
        self.max_age_days = max_age_days
        self.limit = limit
        self.created_at = time.time()

@dataclass
class MemoryAssociation:
    """
    A weighted association between memories

    This represents how memories are connected and the
    strength of those connections for retrieval.
    """
    source_id: MemoryID
    target_id: MemoryID
    association_type: str = "related"               # Type of association
    strength: float = 1.0                          # Association strength
    created_at: float = field(default_factory=time.time)
    context: Dict[str, Any] = field(default_factory=dict)

    def decay(self, decay_rate: float = 0.005):
        """Decay association strength over time"""
        time_factor = (time.time() - self.created_at) / (24 * 3600)
        self.strength = max(0.0, self.strength - decay_rate * time_factor)

class WorkingMemory:
    """
    Working memory for active processing

    This holds information currently being processed or
    manipulated, with very limited capacity but fast access.
    """

    def __init__(self, capacity: int = 7):  # Miller's magic number
        self.capacity = capacity
        self.items: OrderedDict[MemoryID, MemoryItem] = OrderedDict()
        self.total_access_count = 0

        print(f"🧠 Working memory initialized (capacity: {capacity})")

    def store(self, content: Any, context: Dict[str, Any] = None,
              tags: Set[str] = None) -> MemoryID:
        """Store an item in working memory"""
        memory_id = str(uuid.uuid4())

        memory_item = MemoryItem(
            id=memory_id,
            content=content,
            memory_type=MemoryType.WORKING,
            priority=MemoryPriority.HIGH,  # Working memory is high priority
            context=context or {},
            tags=tags or set(),
            source="working_memory"
        )

        # Add to working memory
        self.items[memory_id] = memory_item

        # Evict oldest if over capacity
        while len(self.items) > self.capacity:
            oldest_id, oldest_item = self.items.popitem(last=False)
            print(f"   🔄 Evicted {oldest_id[:8]}... from working memory")

        print(f"   💭 Stored in working memory: {memory_id[:8]}...")
        return memory_id

    def retrieve(self, memory_id: MemoryID) -> Optional[MemoryItem]:
        """Retrieve an item from working memory"""
        if memory_id in self.items:
            item = self.items[memory_id]
            item.access()
            self.total_access_count += 1

            # Move to end (most recently used)
            self.items.move_to_end(memory_id)
            return item
        return None

    def search(self, query: MemoryQuery) -> List[MemoryItem]:
        """Search working memory"""
        results = []

        for item in self.items.values():
            # Check memory type filter
            if item.memory_type not in query.memory_types:
                continue

            # Check strength filter
            if item.strength < query.min_strength:
                continue

            # Check age filter
            if query.max_age_days and item.get_age() > query.max_age_days:
                continue

            # Check content query
            if query.content_query:
                content_str = str(item.content).lower()
                if query.content_query.lower() not in content_str:
                    continue

            # Check tags filter
            if query.tags_filter and not query.tags_filter.issubset(item.tags):
                continue

            item.access()
            results.append(item)

        # Sort by retrieval score
        results.sort(key=lambda x: x.calculate_retrieval_score(query.context_filter),
                    reverse=True)

        return results[:query.limit]

    def get_status(self) -> Dict[str, Any]:
        """Get working memory status"""
        return {
            'capacity': self.capacity,
            'current_items': len(self.items),
            'utilization': len(self.items) / self.capacity,
            'total_accesses': self.total_access_count,
            'items': [
                {
                    'id': item.id[:8] + '...',
                    'type': str(type(item.content).__name__),
                    'strength': item.strength,
                    'accesses': item.access_count
                }
                for item in list(self.items.values())[-5:]  # Last 5 items
            ]
        }

class LongTermMemory:
    """
    Long-term memory storage with different specialized systems

    This provides persistent storage for different types of memories
    with sophisticated retrieval and consolidation mechanisms.
    """

    def __init__(self, storage_path: str = ":memory:"):
        self.storage_path = storage_path
        self.memories: Dict[MemoryID, MemoryItem] = {}
        self.associations: Dict[Tuple[MemoryID, MemoryID], MemoryAssociation] = {}

        # Type-specific indices for fast retrieval
        self.type_indices: Dict[MemoryType, Set[MemoryID]] = {
            mt: set() for mt in MemoryType
        }
        self.tag_index: Dict[str, Set[MemoryID]] = defaultdict(set)
        self.context_index: Dict[str, Dict[str, Set[MemoryID]]] = defaultdict(lambda: defaultdict(set))

        # Persistence (SQLite for production use)
        self.db_connection = None
        self._init_database()

        # Background maintenance
        self._stop_maintenance = threading.Event()
        self._maintenance_thread = threading.Thread(
            target=self._maintenance_loop,
            daemon=True,
            name="LTM-Maintenance"
        )
        self._maintenance_thread.start()

        print(f"🏛️  Long-term memory initialized (storage: {storage_path})")

    def _init_database(self):
        """Initialize SQLite database for persistence"""
        try:
            self.db_connection = sqlite3.connect(
                self.storage_path,
                check_same_thread=False,
                timeout=10.0
            )

            # Create tables
            self.db_connection.execute("""
                CREATE TABLE IF NOT EXISTS memories (
                    id TEXT PRIMARY KEY,
                    content BLOB,
                    memory_type TEXT,
                    priority INTEGER,
                    created_at REAL,
                    last_accessed REAL,
                    access_count INTEGER,
                    strength REAL,
                    consolidation_status TEXT,
                    context TEXT,
                    tags TEXT,
                    emotional_valence REAL,
                    importance_score REAL,
                    source TEXT,
                    confidence REAL
                )
            """)

            self.db_connection.execute("""
                CREATE TABLE IF NOT EXISTS associations (
                    source_id TEXT,
                    target_id TEXT,
                    association_type TEXT,
                    strength REAL,
                    created_at REAL,
                    context TEXT,
                    PRIMARY KEY (source_id, target_id)
                )
            """)

            # Create indices
            self.db_connection.execute("CREATE INDEX IF NOT EXISTS idx_memory_type ON memories(memory_type)")
            self.db_connection.execute("CREATE INDEX IF NOT EXISTS idx_strength ON memories(strength)")
            self.db_connection.execute("CREATE INDEX IF NOT EXISTS idx_last_accessed ON memories(last_accessed)")

            self.db_connection.commit()

            # Load existing memories
            self._load_from_database()

        except Exception as e:
            print(f"⚠️  Database initialization failed: {e}")
            self.db_connection = None

    def _load_from_database(self):
        """Load memories from database"""
        if not self.db_connection:
            return

        try:
            cursor = self.db_connection.execute("SELECT * FROM memories")
            for row in cursor:
                memory_item = self._row_to_memory_item(row)
                self.memories[memory_item.id] = memory_item
                self._update_indices(memory_item)

            # Load associations
            cursor = self.db_connection.execute("SELECT * FROM associations")
            for row in cursor:
                association = self._row_to_association(row)
                key = (association.source_id, association.target_id)
                self.associations[key] = association

            print(f"   📚 Loaded {len(self.memories)} memories from database")

        except Exception as e:
            print(f"⚠️  Failed to load from database: {e}")

    def _row_to_memory_item(self, row) -> MemoryItem:
        """Convert database row to MemoryItem"""
        content = pickle.loads(row[1]) if row[1] else None
        context = json.loads(row[8]) if row[8] else {}
        tags = set(json.loads(row[9])) if row[9] else set()

        return MemoryItem(
            id=row[0],
            content=content,
            memory_type=MemoryType(row[2]),
            priority=MemoryPriority(row[3]),
            created_at=row[4],
            last_accessed=row[5],
            access_count=row[6],
            strength=row[7],
            consolidation_status=ConsolidationStatus(row[10]),
            context=context,
            tags=tags,
            emotional_valence=row[11],
            importance_score=row[12],
            source=row[13],
            confidence=row[14]
        )

    def _row_to_association(self, row) -> MemoryAssociation:
        """Convert database row to MemoryAssociation"""
        context = json.loads(row[5]) if row[5] else {}

        return MemoryAssociation(
            source_id=row[0],
            target_id=row[1],
            association_type=row[2],
            strength=row[3],
            created_at=row[4],
            context=context
        )

    def store(self, content: Any, memory_type: MemoryType = MemoryType.SEMANTIC,
              priority: MemoryPriority = MemoryPriority.NORMAL,
              context: Dict[str, Any] = None, tags: Set[str] = None,
              emotional_valence: float = 0.0, importance_score: float = 0.5,
              source: str = "ltm") -> MemoryID:
        """Store a memory in long-term memory"""
        memory_id = str(uuid.uuid4())

        memory_item = MemoryItem(
            id=memory_id,
            content=content,
            memory_type=memory_type,
            priority=priority,
            context=context or {},
            tags=tags or set(),
            emotional_valence=emotional_valence,
            importance_score=importance_score,
            source=source
        )

        # Store in memory
        self.memories[memory_id] = memory_item
        self._update_indices(memory_item)

        # Persist to database
        self._persist_memory(memory_item)

        print(f"   🏛️  Stored in long-term memory: {memory_id[:8]}... ({memory_type.value})")
        return memory_id

    def retrieve(self, memory_id: MemoryID) -> Optional[MemoryItem]:
        """Retrieve a specific memory"""
        if memory_id in self.memories:
            item = self.memories[memory_id]
            item.access()
            self._persist_memory(item)  # Update access info
            return item
        return None

    def search(self, query: MemoryQuery) -> List[MemoryItem]:
        """Search long-term memory"""
        candidate_ids = set()

        # Use indices for efficient filtering
        if query.memory_types:
            for memory_type in query.memory_types:
                candidate_ids.update(self.type_indices[memory_type])
        else:
            candidate_ids = set(self.memories.keys())

        # Filter by tags if specified
        if query.tags_filter:
            tag_candidates = set()
            for tag in query.tags_filter:
                tag_candidates.update(self.tag_index[tag])
            candidate_ids &= tag_candidates

        # Filter by context if specified
        if query.context_filter:
            context_candidates = set()
            for key, value in query.context_filter.items():
                context_candidates.update(self.context_index[key][str(value)])
            candidate_ids &= context_candidates

        # Apply filters and scoring
        results = []
        for memory_id in candidate_ids:
            item = self.memories[memory_id]

            # Apply filters
            if item.strength < query.min_strength:
                continue

            if query.max_age_days and item.get_age() > query.max_age_days:
                continue

            # Content search
            if query.content_query:
                content_str = str(item.content).lower()
                if query.content_query.lower() not in content_str:
                    continue

            item.access()
            results.append(item)

        # Sort by retrieval score
        results.sort(key=lambda x: x.calculate_retrieval_score(query.context_filter),
                    reverse=True)

        return results[:query.limit]

    def add_association(self, source_id: MemoryID, target_id: MemoryID,
                       association_type: str = "related", strength: float = 1.0,
                       context: Dict[str, Any] = None) -> bool:
        """Add an association between memories"""
        if source_id not in self.memories or target_id not in self.memories:
            return False

        association = MemoryAssociation(
            source_id=source_id,
            target_id=target_id,
            association_type=association_type,
            strength=strength,
            context=context or {}
        )

        key = (source_id, target_id)
        self.associations[key] = association

        # Update memory associations
        self.memories[source_id].add_association(target_id)
        self.memories[target_id].add_association(source_id)

        # Persist association
        self._persist_association(association)

        return True

    def get_associations(self, memory_id: MemoryID,
                        association_type: str = None) -> List[MemoryAssociation]:
        """Get associations for a memory"""
        associations = []

        for (source_id, target_id), association in self.associations.items():
            if source_id == memory_id or target_id == memory_id:
                if association_type is None or association.association_type == association_type:
                    associations.append(association)

        # Sort by strength
        associations.sort(key=lambda x: x.strength, reverse=True)
        return associations

    def _update_indices(self, memory_item: MemoryItem):
        """Update search indices"""
        # Type index
        self.type_indices[memory_item.memory_type].add(memory_item.id)

        # Tag index
        for tag in memory_item.tags:
            self.tag_index[tag].add(memory_item.id)

        # Context index
        for key, value in memory_item.context.items():
            self.context_index[key][str(value)].add(memory_item.id)

    def _persist_memory(self, memory_item: MemoryItem):
        """Persist memory to database"""
        if not self.db_connection:
            return

        try:
            self.db_connection.execute("""
                INSERT OR REPLACE INTO memories VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                memory_item.id,
                pickle.dumps(memory_item.content),
                memory_item.memory_type.value,
                memory_item.priority.value,
                memory_item.created_at,
                memory_item.last_accessed,
                memory_item.access_count,
                memory_item.strength,
                memory_item.consolidation_status.value,
                json.dumps(memory_item.context),
                json.dumps(list(memory_item.tags)),
                memory_item.emotional_valence,
                memory_item.importance_score,
                memory_item.source,
                memory_item.confidence
            ))
            self.db_connection.commit()

        except Exception as e:
            print(f"⚠️  Failed to persist memory: {e}")

    def _persist_association(self, association: MemoryAssociation):
        """Persist association to database"""
        if not self.db_connection:
            return

        try:
            self.db_connection.execute("""
                INSERT OR REPLACE INTO associations VALUES (?, ?, ?, ?, ?, ?)
            """, (
                association.source_id,
                association.target_id,
                association.association_type,
                association.strength,
                association.created_at,
                json.dumps(association.context)
            ))
            self.db_connection.commit()

        except Exception as e:
            print(f"⚠️  Failed to persist association: {e}")

    def _maintenance_loop(self):
        """Background maintenance for memory decay and consolidation"""
        while not self._stop_maintenance.wait(60):  # Run every minute
            try:
                self._apply_decay()
                self._consolidate_memories()
                self._cleanup_weak_associations()

            except Exception as e:
                print(f"⚠️  Memory maintenance error: {e}")

    def _apply_decay(self):
        """Apply decay to all memories"""
        for memory_item in self.memories.values():
            memory_item.decay()

    def _consolidate_memories(self):
        """Move memories through consolidation stages"""
        for memory_item in self.memories.values():
            if (memory_item.consolidation_status == ConsolidationStatus.FRESH and
                memory_item.access_count > 2):
                memory_item.consolidation_status = ConsolidationStatus.REHEARSED

            elif (memory_item.consolidation_status == ConsolidationStatus.REHEARSED and
                  memory_item.get_age() > 1.0 and memory_item.access_count > 5):
                memory_item.consolidation_status = ConsolidationStatus.CONSOLIDATING

            elif (memory_item.consolidation_status == ConsolidationStatus.CONSOLIDATING and
                  memory_item.get_age() > 7.0 and memory_item.strength > 0.7):
                memory_item.consolidation_status = ConsolidationStatus.STABLE

    def _cleanup_weak_associations(self):
        """Remove very weak associations"""
        to_remove = []
        for key, association in self.associations.items():
            association.decay()
            if association.strength < 0.1:
                to_remove.append(key)

        for key in to_remove:
            del self.associations[key]

    def get_status(self) -> Dict[str, Any]:
        """Get long-term memory status"""
        type_counts = {mt.value: len(ids) for mt, ids in self.type_indices.items()}

        consolidation_counts = defaultdict(int)
        for memory in self.memories.values():
            consolidation_counts[memory.consolidation_status.value] += 1

        return {
            'total_memories': len(self.memories),
            'total_associations': len(self.associations),
            'by_type': type_counts,
            'by_consolidation': dict(consolidation_counts),
            'total_tags': len(self.tag_index),
            'database_connected': self.db_connection is not None
        }

    def shutdown(self):
        """Shutdown long-term memory"""
        self._stop_maintenance.set()
        if self.db_connection:
            self.db_connection.close()

class MemorySystem:
    """
    Integrated memory system combining all memory types

    This orchestrates the different memory systems and provides
    a unified interface for memory operations and intelligent routing.
    """

    def __init__(self, working_memory_capacity: int = 7,
                 ltm_storage_path: str = ":memory:"):
        self.working_memory = WorkingMemory(working_memory_capacity)
        self.long_term_memory = LongTermMemory(ltm_storage_path)

        # Memory consolidation queue
        self.consolidation_queue = queue.PriorityQueue()
        self.consolidation_threshold = 5  # Access count to trigger consolidation

        # Background consolidation
        self._stop_consolidation = threading.Event()
        self._consolidation_thread = threading.Thread(
            target=self._consolidation_loop,
            daemon=True,
            name="Memory-Consolidation"
        )
        self._consolidation_thread.start()

        # Memory statistics
        self.consolidation_count = 0
        self.retrieval_count = 0
        self.total_memories = 0

        print("🧠 Integrated memory system initialized")

    def store(self, content: Any, memory_type: MemoryType = MemoryType.SEMANTIC,
              priority: MemoryPriority = MemoryPriority.NORMAL,
              context: Dict[str, Any] = None, tags: Set[str] = None,
              emotional_valence: float = 0.0, importance_score: float = 0.5,
              force_ltm: bool = False) -> MemoryID:
        """
        Store a memory in the appropriate system

        Uses intelligent routing to decide between working memory
        and long-term memory based on content and context.
        """
        self.total_memories += 1

        # Decide storage location
        if (force_ltm or
            priority in [MemoryPriority.CRITICAL, MemoryPriority.HIGH] or
            memory_type in [MemoryType.SEMANTIC, MemoryType.PROCEDURAL] or
            importance_score > 0.7):

            # Store directly in long-term memory
            memory_id = self.long_term_memory.store(
                content=content,
                memory_type=memory_type,
                priority=priority,
                context=context,
                tags=tags,
                emotional_valence=emotional_valence,
                importance_score=importance_score,
                source="direct_ltm"
            )

        else:
            # Store in working memory (might be consolidated later)
            memory_id = self.working_memory.store(
                content=content,
                context=context,
                tags=tags
            )

            # Queue for potential consolidation
            if priority != MemoryPriority.TRANSIENT:
                consolidation_priority = 10 - priority.value  # Higher priority = lower number
                self.consolidation_queue.put((consolidation_priority, time.time(), memory_id))

        return memory_id

    def retrieve(self, memory_id: MemoryID) -> Optional[MemoryItem]:
        """Retrieve a memory from any system"""
        self.retrieval_count += 1

        # Try working memory first (faster)
        memory_item = self.working_memory.retrieve(memory_id)
        if memory_item:
            return memory_item

        # Try long-term memory
        memory_item = self.long_term_memory.retrieve(memory_id)
        if memory_item:
            # Consider promoting to working memory if frequently accessed
            if memory_item.access_count > 3 and memory_item.get_recency() < 0.1:
                self._promote_to_working_memory(memory_item)

        return memory_item

    def search(self, query: MemoryQuery) -> List[MemoryItem]:
        """Search across all memory systems"""
        results = []

        # Search working memory
        working_results = self.working_memory.search(query)
        results.extend(working_results)

        # Search long-term memory
        ltm_results = self.long_term_memory.search(query)
        results.extend(ltm_results)

        # Remove duplicates and sort by relevance
        seen_ids = set()
        unique_results = []
        for item in results:
            if item.id not in seen_ids:
                seen_ids.add(item.id)
                unique_results.append(item)

        # Sort by retrieval score
        unique_results.sort(
            key=lambda x: x.calculate_retrieval_score(query.context_filter),
            reverse=True
        )

        return unique_results[:query.limit]

    def associate(self, memory_id1: MemoryID, memory_id2: MemoryID,
                 association_type: str = "related", strength: float = 1.0,
                 context: Dict[str, Any] = None) -> bool:
        """Create an association between memories"""
        # Associations are stored in long-term memory
        return self.long_term_memory.add_association(
            memory_id1, memory_id2, association_type, strength, context
        )

    def get_related_memories(self, memory_id: MemoryID,
                           association_type: str = None,
                           max_results: int = 10) -> List[MemoryItem]:
        """Get memories related to a given memory"""
        associations = self.long_term_memory.get_associations(memory_id, association_type)

        related_memories = []
        for association in associations[:max_results]:
            # Get the other memory in the association
            other_id = (association.target_id if association.source_id == memory_id
                       else association.source_id)

            other_memory = self.retrieve(other_id)
            if other_memory:
                related_memories.append(other_memory)

        return related_memories

    def consolidate_memory(self, memory_id: MemoryID) -> bool:
        """Manually consolidate a memory from working to long-term"""
        memory_item = self.working_memory.retrieve(memory_id)
        if not memory_item:
            return False

        # Store in long-term memory
        ltm_id = self.long_term_memory.store(
            content=memory_item.content,
            memory_type=MemoryType.EPISODIC,  # Working memories become episodic
            priority=memory_item.priority,
            context=memory_item.context,
            tags=memory_item.tags,
            source="consolidated_from_wm"
        )

        # Remove from working memory
        if memory_id in self.working_memory.items:
            del self.working_memory.items[memory_id]

        self.consolidation_count += 1
        print(f"   🔄 Consolidated memory {memory_id[:8]}... → {ltm_id[:8]}...")
        return True

    def _promote_to_working_memory(self, memory_item: MemoryItem):
        """Promote a frequently accessed LTM item to working memory"""
        # Create a copy in working memory
        self.working_memory.store(
            content=memory_item.content,
            context=memory_item.context,
            tags=memory_item.tags
        )
        print(f"   ⬆️  Promoted {memory_item.id[:8]}... to working memory")

    def _consolidation_loop(self):
        """Background consolidation loop"""
        while not self._stop_consolidation.wait(5):  # Check every 5 seconds
            try:
                # Process consolidation queue
                while not self.consolidation_queue.empty():
                    try:
                        priority, timestamp, memory_id = self.consolidation_queue.get_nowait()

                        # Check if memory still exists and meets consolidation criteria
                        memory_item = self.working_memory.retrieve(memory_id)
                        if (memory_item and
                            memory_item.access_count >= self.consolidation_threshold):
                            self.consolidate_memory(memory_id)

                    except queue.Empty:
                        break

            except Exception as e:
                print(f"⚠️  Consolidation error: {e}")

    def get_memory_statistics(self) -> Dict[str, Any]:
        """Get comprehensive memory statistics"""
        wm_status = self.working_memory.get_status()
        ltm_status = self.long_term_memory.get_status()

        return {
            'working_memory': wm_status,
            'long_term_memory': ltm_status,
            'consolidation_count': self.consolidation_count,
            'retrieval_count': self.retrieval_count,
            'total_memories_created': self.total_memories,
            'consolidation_queue_size': self.consolidation_queue.qsize()
        }

    def shutdown(self):
        """Shutdown the memory system"""
        self._stop_consolidation.set()
        self.long_term_memory.shutdown()
        print("🛑 Memory system shutdown complete")

class MemoryAwareAgent:
    """
    An agent with sophisticated memory capabilities

    This demonstrates how to integrate memory systems into agents
    for context-aware processing and learning from experience.
    """

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 memory_system: MemorySystem = None,
                 memory_capacity: int = 7):
        self.id = agent_id or str(uuid.uuid4())
        self.name = name or f"memory_agent_{self.id[:8]}"

        # Memory system
        if memory_system:
            self.memory = memory_system
        else:
            self.memory = MemorySystem(
                working_memory_capacity=memory_capacity,
                ltm_storage_path=f"agent_{self.id}_memory.db"
            )

        # Agent state
        self.current_context: Dict[str, Any] = {}
        self.conversation_history: List[MemoryID] = []
        self.learned_patterns: Dict[str, Any] = {}

        # Performance tracking
        self.tasks_processed = 0
        self.learning_events = 0
        self.context_switches = 0

        print(f"🧠 Created memory-aware agent: {self.name}")

    def process_with_memory(self, input_data: Any, context: Dict[str, Any] = None) -> Any:
        """Process input using memory-enhanced reasoning"""
        self.tasks_processed += 1
        processing_context = context or {}

        # Store current input in working memory
        input_memory_id = self.memory.store(
            content=input_data,
            memory_type=MemoryType.WORKING,
            context=processing_context,
            tags={"input", "current"}
        )

        # Retrieve relevant memories
        relevant_memories = self._get_relevant_context(input_data, processing_context)

        # Update current context
        self._update_context(processing_context, relevant_memories)

        # Process with memory-enhanced reasoning
        result = self._memory_enhanced_processing(input_data, relevant_memories)

        # Store result and create associations
        result_memory_id = self.memory.store(
            content=result,
            memory_type=MemoryType.EPISODIC,
            context=processing_context,
            tags={"output", "result"},
            importance_score=0.6
        )

        # Associate input and output
        self.memory.associate(input_memory_id, result_memory_id, "input_output")

        # Associate with relevant memories
        for memory in relevant_memories[:3]:  # Top 3 most relevant
            self.memory.associate(result_memory_id, memory.id, "contextual")

        # Add to conversation history
        self.conversation_history.append(result_memory_id)

        # Learn from this interaction
        self._learn_from_interaction(input_data, result, relevant_memories)

        return result

    def _get_relevant_context(self, input_data: Any, context: Dict[str, Any]) -> List[MemoryItem]:
        """Retrieve memories relevant to current input"""
        # Create search query based on input
        content_query = str(input_data)[:100]  # First 100 chars

        query = MemoryQuery(
            content_query=content_query,
            memory_types=[MemoryType.EPISODIC, MemoryType.SEMANTIC, MemoryType.PROCEDURAL],
            context_filter=context,
            min_strength=0.3,
            max_age_days=30,  # Focus on recent memories
            limit=10
        )

        return self.memory.search(query)

    def _update_context(self, current_context: Dict[str, Any],
                       relevant_memories: List[MemoryItem]):
        """Update agent's current context based on retrieved memories"""
        if self.current_context != current_context:
            self.context_switches += 1

        self.current_context = current_context.copy()

        # Add memory-derived context
        for memory in relevant_memories[:5]:
            for key, value in memory.context.items():
                if key not in self.current_context:
                    self.current_context[f"memory_{key}"] = value

    def _memory_enhanced_processing(self, input_data: Any,
                                  relevant_memories: List[MemoryItem]) -> Any:
        """Process input with memory context"""
        # Basic processing (would be more sophisticated in real implementation)
        base_result = f"Processed: {input_data}"

        # Enhance with memory context
        if relevant_memories:
            memory_insights = []
            for memory in relevant_memories[:3]:
                if hasattr(memory.content, '__str__'):
                    memory_insights.append(f"Recalled: {str(memory.content)[:50]}...")

            if memory_insights:
                base_result += f" | Context: {'; '.join(memory_insights)}"

        # Add learned patterns
        if self.learned_patterns:
            pattern_match = self._find_pattern_match(input_data)
            if pattern_match:
                base_result += f" | Pattern: {pattern_match}"

        return base_result

    def _learn_from_interaction(self, input_data: Any, result: Any,
                              relevant_memories: List[MemoryItem]):
        """Learn patterns from this interaction"""
        self.learning_events += 1

        # Simple pattern learning (extract common elements)
        input_str = str(input_data).lower()

        # Learn input patterns
        input_words = input_str.split()
        for word in input_words:
            if len(word) > 3:  # Meaningful words
                pattern_key = f"input_pattern_{word}"
                if pattern_key not in self.learned_patterns:
                    self.learned_patterns[pattern_key] = {
                        'count': 0,
                        'contexts': [],
                        'results': []
                    }

                pattern = self.learned_patterns[pattern_key]
                pattern['count'] += 1
                pattern['contexts'].append(self.current_context.copy())
                pattern['results'].append(str(result)[:100])

                # Keep only recent examples
                if len(pattern['contexts']) > 10:
                    pattern['contexts'] = pattern['contexts'][-10:]
                    pattern['results'] = pattern['results'][-10:]

        # Store learning as semantic memory
        if self.learning_events % 10 == 0:  # Every 10 interactions
            learning_summary = {
                'agent_id': self.id,
                'patterns_learned': len(self.learned_patterns),
                'interaction_count': self.learning_events,
                'recent_patterns': list(self.learned_patterns.keys())[-5:]
            }

            self.memory.store(
                content=learning_summary,
                memory_type=MemoryType.SEMANTIC,
                priority=MemoryPriority.HIGH,
                tags={"learning", "patterns", "self_knowledge"},
                importance_score=0.8
            )

    def _find_pattern_match(self, input_data: Any) -> Optional[str]:
        """Find matching learned patterns"""
        input_str = str(input_data).lower()

        for pattern_key, pattern_data in self.learned_patterns.items():
            if pattern_data['count'] > 2:  # Pattern seen multiple times
                pattern_word = pattern_key.replace('input_pattern_', '')
                if pattern_word in input_str:
                    return f"{pattern_word} (seen {pattern_data['count']} times)"

        return None

    def recall_conversation(self, limit: int = 10) -> List[MemoryItem]:
        """Recall recent conversation history"""
        recent_memory_ids = self.conversation_history[-limit:]

        memories = []
        for memory_id in recent_memory_ids:
            memory = self.memory.retrieve(memory_id)
            if memory:
                memories.append(memory)

        return memories

    def get_learning_summary(self) -> Dict[str, Any]:
        """Get summary of what the agent has learned"""
        pattern_summary = {}
        for pattern_key, pattern_data in self.learned_patterns.items():
            if pattern_data['count'] > 1:  # Only meaningful patterns
                pattern_summary[pattern_key] = {
                    'frequency': pattern_data['count'],
                    'example_result': pattern_data['results'][-1] if pattern_data['results'] else None
                }

        return {
            'total_patterns': len(self.learned_patterns),
            'meaningful_patterns': len(pattern_summary),
            'pattern_details': pattern_summary,
            'learning_events': self.learning_events,
            'context_switches': self.context_switches,
            'conversation_length': len(self.conversation_history)
        }

    def get_status(self) -> Dict[str, Any]:
        """Get agent status including memory statistics"""
        memory_stats = self.memory.get_memory_statistics()
        learning_summary = self.get_learning_summary()

        return {
            'id': self.id,
            'name': self.name,
            'tasks_processed': self.tasks_processed,
            'current_context_keys': list(self.current_context.keys()),
            'memory_statistics': memory_stats,
            'learning_summary': learning_summary
        }

class KnowledgeGraph:
    """
    Knowledge graph for semantic memory organization

    This provides structured knowledge representation with
    entities, relationships, and semantic reasoning capabilities.
    """

    def __init__(self):
        self.entities: Dict[str, Dict[str, Any]] = {}
        self.relationships: Dict[str, List[Tuple[str, str, Dict[str, Any]]]] = defaultdict(list)
        self.entity_types: Dict[str, str] = {}

        # Indices for efficient querying
        self.type_index: Dict[str, Set[str]] = defaultdict(set)
        self.attribute_index: Dict[str, Dict[str, Set[str]]] = defaultdict(lambda: defaultdict(set))

        print("🕸️  Knowledge graph initialized")

    def add_entity(self, entity_id: str, entity_type: str,
                  attributes: Dict[str, Any] = None) -> bool:
        """Add an entity to the knowledge graph"""
        if entity_id in self.entities:
            return False

        self.entities[entity_id] = attributes or {}
        self.entity_types[entity_id] = entity_type
        self.type_index[entity_type].add(entity_id)

        # Update attribute indices
        for attr_name, attr_value in (attributes or {}).items():
            self.attribute_index[attr_name][str(attr_value)].add(entity_id)

        print(f"   🏷️  Added entity: {entity_id} ({entity_type})")
        return True

    def add_relationship(self, source_id: str, relationship_type: str, target_id: str,
                        properties: Dict[str, Any] = None) -> bool:
        """Add a relationship between entities"""
        if source_id not in self.entities or target_id not in self.entities:
            return False

        relationship = (source_id, target_id, properties or {})
        self.relationships[relationship_type].append(relationship)

        print(f"   🔗 Added relationship: {source_id} --{relationship_type}--> {target_id}")
        return True

    def query_entities(self, entity_type: str = None,
                      attributes: Dict[str, Any] = None) -> List[str]:
        """Query entities by type and attributes"""
        candidates = set()

        if entity_type:
            candidates = self.type_index[entity_type].copy()
        else:
            candidates = set(self.entities.keys())

        # Filter by attributes
        if attributes:
            for attr_name, attr_value in attributes.items():
                attr_candidates = self.attribute_index[attr_name][str(attr_value)]
                candidates &= attr_candidates

        return list(candidates)

    def query_relationships(self, relationship_type: str,
                          source_id: str = None, target_id: str = None) -> List[Tuple[str, str, Dict[str, Any]]]:
        """Query relationships"""
        relationships = self.relationships[relationship_type]

        if source_id or target_id:
            filtered = []
            for src, tgt, props in relationships:
                if (source_id is None or src == source_id) and \
                   (target_id is None or tgt == target_id):
                    filtered.append((src, tgt, props))
            return filtered

        return relationships

    def get_neighbors(self, entity_id: str, relationship_type: str = None) -> List[str]:
        """Get neighboring entities"""
        neighbors = set()

        # Check all relationship types if none specified
        rel_types = [relationship_type] if relationship_type else self.relationships.keys()

        for rel_type in rel_types:
            for src, tgt, props in self.relationships[rel_type]:
                if src == entity_id:
                    neighbors.add(tgt)
                elif tgt == entity_id:
                    neighbors.add(src)

        return list(neighbors)

    def get_path(self, start_id: str, end_id: str, max_depth: int = 3) -> Optional[List[str]]:
        """Find shortest path between entities"""
        if start_id not in self.entities or end_id not in self.entities:
            return None

        # BFS to find shortest path
        queue = deque([(start_id, [start_id])])
        visited = {start_id}

        while queue:
            current_id, path = queue.popleft()

            if len(path) > max_depth:
                continue

            if current_id == end_id:
                return path

            for neighbor in self.get_neighbors(current_id):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path + [neighbor]))

        return None

    def get_statistics(self) -> Dict[str, Any]:
        """Get knowledge graph statistics"""
        total_relationships = sum(len(rels) for rels in self.relationships.values())

        return {
            'total_entities': len(self.entities),
            'total_relationships': total_relationships,
            'entity_types': len(self.type_index),
            'relationship_types': len(self.relationships),
            'entity_type_distribution': {
                et: len(entities) for et, entities in self.type_index.items()
            },
            'relationship_type_distribution': {
                rt: len(rels) for rt, rels in self.relationships.items()
            }
        }

class MemoryVisualizer:
    """
    Visualization tools for memory systems

    This provides debugging and analysis capabilities for
    understanding memory system behavior and performance.
    """

    def __init__(self, memory_system: MemorySystem):
        self.memory_system = memory_system

        print("📊 Memory visualizer initialized")

    def create_memory_dashboard(self) -> Dict[str, Any]:
        """Create a comprehensive memory dashboard"""
        stats = self.memory_system.get_memory_statistics()

        # Memory distribution analysis
        ltm_by_type = stats['long_term_memory']['by_type']
        total_ltm = sum(ltm_by_type.values())

        type_percentages = {}
        for mem_type, count in ltm_by_type.items():
            percentage = (count / max(total_ltm, 1)) * 100
            type_percentages[mem_type] = percentage

        # Consolidation analysis
        consolidation_by_status = stats['long_term_memory']['by_consolidation']
        total_consolidated = sum(consolidation_by_status.values())

        consolidation_percentages = {}
        for status, count in consolidation_by_status.items():
            percentage = (count / max(total_consolidated, 1)) * 100
            consolidation_percentages[status] = percentage

        # Working memory efficiency
        wm_stats = stats['working_memory']
        wm_efficiency = {
            'utilization': wm_stats['utilization'] * 100,
            'avg_accesses_per_item': wm_stats['total_accesses'] / max(wm_stats['current_items'], 1)
        }

        return {
            'overview': {
                'total_memories': stats['total_memories_created'],
                'consolidations': stats['consolidation_count'],
                'retrievals': stats['retrieval_count'],
                'consolidation_rate': stats['consolidation_count'] / max(stats['total_memories_created'], 1)
            },
            'memory_distribution': {
                'by_type_counts': ltm_by_type,
                'by_type_percentages': type_percentages
            },
            'consolidation_analysis': {
                'by_status_counts': consolidation_by_status,
                'by_status_percentages': consolidation_percentages
            },
            'working_memory_efficiency': wm_efficiency,
            'system_health': {
                'consolidation_queue_size': stats['consolidation_queue_size'],
                'ltm_database_connected': stats['long_term_memory']['database_connected']
            }
        }

    def analyze_memory_patterns(self, limit: int = 100) -> Dict[str, Any]:
        """Analyze patterns in memory usage"""
        # Get recent memories for analysis
        query = MemoryQuery(
            memory_types=list(MemoryType),
            limit=limit
        )

        recent_memories = self.memory_system.search(query)

        if not recent_memories:
            return {'error': 'No memories found for analysis'}

        # Analyze patterns
        type_access_patterns = defaultdict(list)
        strength_distribution = defaultdict(int)
        age_distribution = defaultdict(int)

        for memory in recent_memories:
            type_access_patterns[memory.memory_type.value].append(memory.access_count)

            # Strength distribution (bins)
            strength_bin = int(memory.strength * 10) / 10
            strength_distribution[strength_bin] += 1

            # Age distribution (days)
            age_days = int(memory.get_age())
            age_distribution[age_days] += 1

        # Calculate averages
        avg_access_by_type = {}
        for mem_type, accesses in type_access_patterns.items():
            avg_access_by_type[mem_type] = sum(accesses) / len(accesses)

        return {
            'sample_size': len(recent_memories),
            'average_access_by_type': avg_access_by_type,
            'strength_distribution': dict(strength_distribution),
            'age_distribution': dict(age_distribution),
            'most_accessed_memory': max(recent_memories, key=lambda m: m.access_count),
            'strongest_memory': max(recent_memories, key=lambda m: m.strength)
        }

    def generate_memory_report(self) -> str:
        """Generate a human-readable memory report"""
        dashboard = self.create_memory_dashboard()
        patterns = self.analyze_memory_patterns()

        report = []
        report.append("=" * 60)
        report.append("MEMORY SYSTEM ANALYSIS REPORT")
        report.append("=" * 60)
        report.append("")

        # Overview
        overview = dashboard['overview']
        report.append("📊 SYSTEM OVERVIEW")
        report.append(f"   Total memories created: {overview['total_memories']}")
        report.append(f"   Successful consolidations: {overview['consolidations']}")
        report.append(f"   Total retrievals: {overview['retrievals']}")
        report.append(f"   Consolidation rate: {overview['consolidation_rate']:.1%}")
        report.append("")

        # Memory distribution
        distribution = dashboard['memory_distribution']
        report.append("🧠 MEMORY TYPE DISTRIBUTION")
        for mem_type, percentage in distribution['by_type_percentages'].items():
            count = distribution['by_type_counts'][mem_type]
            report.append(f"   {mem_type.capitalize()}: {count} memories ({percentage:.1f}%)")
        report.append("")

        # Working memory efficiency
        wm_eff = dashboard['working_memory_efficiency']
        report.append("💭 WORKING MEMORY EFFICIENCY")
        report.append(f"   Utilization: {wm_eff['utilization']:.1f}%")
        report.append(f"   Average accesses per item: {wm_eff['avg_accesses_per_item']:.1f}")
        report.append("")

        # Consolidation analysis
        consolidation = dashboard['consolidation_analysis']
        report.append("🔄 CONSOLIDATION STATUS")
        for status, percentage in consolidation['by_status_percentages'].items():
            count = consolidation['by_status_counts'][status]
            report.append(f"   {status.capitalize()}: {count} memories ({percentage:.1f}%)")
        report.append("")

        # Pattern analysis
        if 'error' not in patterns:
            report.append("📈 USAGE PATTERNS")
            report.append(f"   Sample size: {patterns['sample_size']} memories")
            report.append("   Average access by type:")
            for mem_type, avg_access in patterns['average_access_by_type'].items():
                report.append(f"     {mem_type.capitalize()}: {avg_access:.1f} accesses")

            most_accessed = patterns['most_accessed_memory']
            report.append(f"   Most accessed memory: {most_accessed.access_count} accesses")

            strongest = patterns['strongest_memory']
            report.append(f"   Strongest memory: {strongest.strength:.2f} strength")

        report.append("")
        report.append("=" * 60)

        return "\n".join(report)

# =============================================================================
# INITIALIZATION COMPLETE
# =============================================================================

print("🔧 Tutorial 10 initialization complete!")
print("✅ All classes loaded successfully:")
print("   - MemoryItem for individual memory storage")
print("   - WorkingMemory for short-term active processing")
print("   - LongTermMemory for persistent storage with SQLite")
print("   - MemorySystem for integrated memory management")
print("   - MemoryAwareAgent for intelligent memory-driven processing")
print("   - KnowledgeGraph for semantic relationship modeling")
print("   - MemoryVisualizer for analysis and debugging")
print()
print("🚀 Ready to build memory-enhanced intelligent agents!")
print()

🔧 Tutorial 10 initialization complete!
✅ All classes loaded successfully:
   - MemoryItem for individual memory storage
   - WorkingMemory for short-term active processing
   - LongTermMemory for persistent storage with SQLite
   - MemorySystem for integrated memory management
   - MemoryAwareAgent for intelligent memory-driven processing
   - KnowledgeGraph for semantic relationship modeling
   - MemoryVisualizer for analysis and debugging

🚀 Ready to build memory-enhanced intelligent agents!



In [4]:
# DEMO SECTION: Let's build sophisticated memory systems!
# =============================================================================

print("=" * 60)
print("🚀 Tutorial 10: Memory Systems - Multi-Layer Information Storage")
print("=" * 60)
print()

🚀 Tutorial 10: Memory Systems - Multi-Layer Information Storage



In [5]:
# Step 1: Create and test basic memory components
print("📝 Step 1: Creating and testing basic memory components...")

# Create working memory
working_memory = WorkingMemory(capacity=5)

# Test working memory with different types of content
test_data = [
    "Remember this important fact",
    {"user": "Alice", "preference": "coffee"},
    ["item1", "item2", "item3"],
    42,
    "Another piece of information"
]

wm_ids = []
for i, data in enumerate(test_data):
    memory_id = working_memory.store(
        content=data,
        context={"session": "demo", "step": i},
        tags={"test", f"item_{i}"}
    )
    wm_ids.append(memory_id)

print(f"   ✅ Stored {len(wm_ids)} items in working memory")

# Test retrieval
retrieved = working_memory.retrieve(wm_ids[0])
if retrieved:
    print(f"   ✅ Retrieved: {type(retrieved.content).__name__}")

# Test working memory status
wm_status = working_memory.get_status()
print(f"   Working memory utilization: {wm_status['utilization']:.1%}")
print()


📝 Step 1: Creating and testing basic memory components...
🧠 Working memory initialized (capacity: 5)
   💭 Stored in working memory: 7311971a...
   💭 Stored in working memory: c3b54eed...
   💭 Stored in working memory: 266cd44f...
   💭 Stored in working memory: 8171ccb3...
   💭 Stored in working memory: 1b7c300d...
   ✅ Stored 5 items in working memory
   ✅ Retrieved: str
   Working memory utilization: 100.0%



In [6]:
# Step 2: Create and test long-term memory
print("📝 Step 2: Creating and testing long-term memory...")

ltm = LongTermMemory(storage_path="demo_memory.db")

# Store different types of memories
semantic_facts = [
    "Paris is the capital of France",
    "Python is a programming language",
    "Machine learning uses neural networks"
]

episodic_memories = [
    "Had a meeting with the team at 2 PM",
    "Learned about memory systems today",
    "Successfully implemented a new feature"
]

procedural_knowledge = [
    "To start a car: 1) Insert key, 2) Turn ignition, 3) Press gas",
    "To make coffee: 1) Add water, 2) Add grounds, 3) Brew",
    "To debug code: 1) Read error, 2) Check syntax, 3) Test fix"
]

# Store semantic memories
semantic_ids = []
for fact in semantic_facts:
    memory_id = ltm.store(
        content=fact,
        memory_type=MemoryType.SEMANTIC,
        priority=MemoryPriority.HIGH,
        context={"domain": "general_knowledge"},
        tags={"fact", "knowledge"},
        importance_score=0.8
    )
    semantic_ids.append(memory_id)

# Store episodic memories
episodic_ids = []
for episode in episodic_memories:
    memory_id = ltm.store(
        content=episode,
        memory_type=MemoryType.EPISODIC,
        priority=MemoryPriority.NORMAL,
        context={"date": "2025-05-24", "session": "demo"},
        tags={"experience", "personal"},
        emotional_valence=0.3,
        importance_score=0.6
    )
    episodic_ids.append(memory_id)

# Store procedural knowledge
procedural_ids = []
for procedure in procedural_knowledge:
    memory_id = ltm.store(
        content=procedure,
        memory_type=MemoryType.PROCEDURAL,
        priority=MemoryPriority.HIGH,
        context={"type": "instruction"},
        tags={"procedure", "skill"},
        importance_score=0.9
    )
    procedural_ids.append(memory_id)

print(f"   ✅ Stored {len(semantic_ids)} semantic memories")
print(f"   ✅ Stored {len(episodic_ids)} episodic memories")
print(f"   ✅ Stored {len(procedural_ids)} procedural memories")

# Test memory associations
ltm.add_association(
    semantic_ids[1],  # Python programming
    procedural_ids[2],  # Debugging procedure
    "related_skill",
    strength=0.8
)

print("   ✅ Created memory associations")

# Test LTM search
search_query = MemoryQuery(
    content_query="programming",
    memory_types=[MemoryType.SEMANTIC, MemoryType.PROCEDURAL],
    limit=5
)

search_results = ltm.search(search_query)
print(f"   ✅ Search found {len(search_results)} relevant memories")

ltm_status = ltm.get_status()
print(f"   LTM total memories: {ltm_status['total_memories']}")
print()

📝 Step 2: Creating and testing long-term memory...
   📚 Loaded 0 memories from database
🏛️  Long-term memory initialized (storage: demo_memory.db)
   🏛️  Stored in long-term memory: bd3dd89f... (semantic)
   🏛️  Stored in long-term memory: 5b1ef827... (semantic)
   🏛️  Stored in long-term memory: b9b32520... (semantic)
   🏛️  Stored in long-term memory: f52c8999... (episodic)
   🏛️  Stored in long-term memory: 88a1db5c... (episodic)
   🏛️  Stored in long-term memory: 8305e5ba... (episodic)
   🏛️  Stored in long-term memory: 24b1e71f... (procedural)
   🏛️  Stored in long-term memory: 92f0c601... (procedural)
   🏛️  Stored in long-term memory: 2d97d863... (procedural)
   ✅ Stored 3 semantic memories
   ✅ Stored 3 episodic memories
   ✅ Stored 3 procedural memories
   ✅ Created memory associations
   ✅ Search found 1 relevant memories
   LTM total memories: 9



In [7]:
# Step 3: Create integrated memory system
print("📝 Step 3: Creating integrated memory system...")

memory_system = MemorySystem(
    working_memory_capacity=7,
    ltm_storage_path="integrated_memory.db"
)

# Test intelligent memory routing
test_inputs = [
    ("Remember my name is Alice", MemoryType.EPISODIC, MemoryPriority.HIGH),
    ("The sky is blue", MemoryType.SEMANTIC, MemoryPriority.NORMAL),
    ("Current task: process documents", MemoryType.WORKING, MemoryPriority.NORMAL),
    ("How to make tea: boil water, add tea, steep", MemoryType.PROCEDURAL, MemoryPriority.HIGH),
    ("Feeling excited about this project", MemoryType.EMOTIONAL, MemoryPriority.NORMAL)
]

stored_memory_ids = []
for content, mem_type, priority in test_inputs:
    memory_id = memory_system.store(
        content=content,
        memory_type=mem_type,
        priority=priority,
        context={"demo": True, "step": 3},
        tags={"integrated_test"}
    )
    stored_memory_ids.append(memory_id)

print(f"   ✅ Stored {len(stored_memory_ids)} memories with intelligent routing")

# Test cross-memory search
unified_query = MemoryQuery(
    content_query="task",
    memory_types=list(MemoryType),
    context_filter={"demo": True},
    limit=10
)

unified_results = memory_system.search(unified_query)
print(f"   ✅ Unified search found {len(unified_results)} memories across all systems")

# Test memory associations
if len(stored_memory_ids) >= 2:
    memory_system.associate(
        stored_memory_ids[0],  # Alice
        stored_memory_ids[4],  # Excitement
        "emotional_connection",
        strength=0.7
    )
    print("   ✅ Created cross-memory associations")

# Get initial memory statistics
initial_stats = memory_system.get_memory_statistics()
print(f"   Initial system state: {initial_stats['total_memories_created']} total memories")
print()

📝 Step 3: Creating integrated memory system...
🧠 Working memory initialized (capacity: 7)
   📚 Loaded 0 memories from database
🏛️  Long-term memory initialized (storage: integrated_memory.db)
🧠 Integrated memory system initialized
   🏛️  Stored in long-term memory: 8ffa4140... (episodic)
   🏛️  Stored in long-term memory: 4cf0a24e... (semantic)
   💭 Stored in working memory: dbdd9bcd...
   🏛️  Stored in long-term memory: ab382412... (procedural)
   💭 Stored in working memory: 7926f7e2...
   ✅ Stored 5 memories with intelligent routing
   ✅ Unified search found 1 memories across all systems
   ✅ Created cross-memory associations
   Initial system state: 5 total memories



In [8]:
# Step 4: Create and test memory-aware agent
print("📝 Step 4: Creating and testing memory-aware agent...")

# Create memory-aware agent
agent = MemoryAwareAgent(
    name="demo_agent",
    memory_system=memory_system
)

# Test memory-enhanced processing
test_conversations = [
    "Hello, I'm working on a Python project",
    "I need help with debugging my code",
    "Can you help me make some tea?",
    "I'm feeling stuck on this programming task",
    "What did we talk about earlier regarding Python?"
]

print("   Testing conversational memory...")
conversation_results = []
for i, message in enumerate(test_conversations):
    print(f"     User: {message}")

    response = agent.process_with_memory(
        input_data=message,
        context={"conversation_turn": i, "user": "demo_user"}
    )

    conversation_results.append(response)
    print(f"     Agent: {response[:100]}{'...' if len(response) > 100 else ''}")
    print()

# Analyze learning
learning_summary = agent.get_learning_summary()
print(f"   ✅ Agent learned {learning_summary['meaningful_patterns']} meaningful patterns")
print(f"   ✅ Processed {learning_summary['learning_events']} learning events")

# Test conversation recall
print("   Testing conversation recall...")
recent_conversation = agent.recall_conversation(limit=3)
print(f"   ✅ Recalled {len(recent_conversation)} recent exchanges")

# Test related memory retrieval
if stored_memory_ids:
    related_memories = memory_system.get_related_memories(stored_memory_ids[0], max_results=3)
    print(f"   ✅ Found {len(related_memories)} related memories")

print()


📝 Step 4: Creating and testing memory-aware agent...
🧠 Created memory-aware agent: demo_agent
   Testing conversational memory...
     User: Hello, I'm working on a Python project
   💭 Stored in working memory: 08293a24...
   💭 Stored in working memory: cc6cebef...
     Agent: Processed: Hello, I'm working on a Python project

     User: I need help with debugging my code
   💭 Stored in working memory: 32ce9db6...
   💭 Stored in working memory: 878b5036...
     Agent: Processed: I need help with debugging my code

     User: Can you help me make some tea?
   💭 Stored in working memory: 73d5a60e...
   🔄 Evicted dbdd9bcd... from working memory
   💭 Stored in working memory: 31640285...
     Agent: Processed: Can you help me make some tea?

     User: I'm feeling stuck on this programming task
   🔄 Evicted 7926f7e2... from working memory
   💭 Stored in working memory: 19839652...
   🔄 Evicted 08293a24... from working memory
   💭 Stored in working memory: a7c5af59...
     Agent: Processed:

In [9]:
# Step 5: Create and test knowledge graph
print("📝 Step 5: Creating and testing knowledge graph...")

knowledge_graph = KnowledgeGraph()

# Add entities
entities = [
    ("python", "programming_language", {"created": 1991, "type": "interpreted"}),
    ("alice", "person", {"role": "developer", "experience": "senior"}),
    ("debugging", "skill", {"difficulty": "intermediate", "importance": "high"}),
    ("machine_learning", "field", {"complexity": "high", "growth": "rapid"}),
    ("neural_networks", "technology", {"type": "ai", "applications": "many"})
]

for entity_id, entity_type, attributes in entities:
    knowledge_graph.add_entity(entity_id, entity_type, attributes)

# Add relationships
relationships = [
    ("alice", "knows", "python", {"proficiency": "expert"}),
    ("alice", "uses", "debugging", {"frequency": "daily"}),
    ("python", "supports", "machine_learning", {"via": "libraries"}),
    ("machine_learning", "uses", "neural_networks", {"commonly": True}),
    ("debugging", "required_for", "python", {"always": True})
]

for source, rel_type, target, properties in relationships:
    knowledge_graph.add_relationship(source, rel_type, target, properties)

print(f"   ✅ Created knowledge graph with {len(entities)} entities and {len(relationships)} relationships")

# Test knowledge graph queries
python_related = knowledge_graph.get_neighbors("python")
print(f"   ✅ Found {len(python_related)} entities related to Python: {python_related}")

# Test path finding
path = knowledge_graph.get_path("alice", "neural_networks")
if path:
    print(f"   ✅ Found path from Alice to Neural Networks: {' -> '.join(path)}")

kg_stats = knowledge_graph.get_statistics()
print(f"   Knowledge graph stats: {kg_stats['total_entities']} entities, {kg_stats['total_relationships']} relationships")
print()

📝 Step 5: Creating and testing knowledge graph...
🕸️  Knowledge graph initialized
   🏷️  Added entity: python (programming_language)
   🏷️  Added entity: alice (person)
   🏷️  Added entity: debugging (skill)
   🏷️  Added entity: machine_learning (field)
   🏷️  Added entity: neural_networks (technology)
   🔗 Added relationship: alice --knows--> python
   🔗 Added relationship: alice --uses--> debugging
   🔗 Added relationship: python --supports--> machine_learning
   🔗 Added relationship: machine_learning --uses--> neural_networks
   🔗 Added relationship: debugging --required_for--> python
   ✅ Created knowledge graph with 5 entities and 5 relationships
   ✅ Found 3 entities related to Python: ['alice', 'machine_learning', 'debugging']
   Knowledge graph stats: 5 entities, 5 relationships



In [10]:
# Step 6: Memory system performance testing
print("📝 Step 6: Testing memory system performance...")

# Generate load for performance testing
import random

print("   Generating memory load for performance testing...")
performance_test_ids = []

# Create diverse memory content
content_templates = [
    "User {user} performed action {action} at time {time}",
    "System learned pattern: {pattern} with confidence {confidence}",
    "Knowledge fact: {subject} has property {property} with value {value}",
    "Procedure step {step}: {instruction}",
    "Emotional event: feeling {emotion} about {subject}"
]

users = ["Alice", "Bob", "Charlie", "Diana", "Eve"]
actions = ["login", "search", "create", "update", "delete"]
patterns = ["sequence_A", "behavior_B", "trend_C", "anomaly_D"]
subjects = ["system", "user", "data", "process", "interface"]
emotions = ["happy", "frustrated", "excited", "confused", "satisfied"]

# Generate 50 diverse memories
for i in range(50):
    template = random.choice(content_templates)

    if "User" in template:
        content = template.format(
            user=random.choice(users),
            action=random.choice(actions),
            time=f"2025-05-24T{random.randint(9, 17):02d}:{random.randint(0, 59):02d}"
        )
        mem_type = MemoryType.EPISODIC
    elif "learned pattern" in template:
        content = template.format(
            pattern=random.choice(patterns),
            confidence=f"{random.uniform(0.6, 0.9):.2f}"
        )
        mem_type = MemoryType.SEMANTIC
    elif "Knowledge fact" in template:
        content = template.format(
            subject=random.choice(subjects),
            property=random.choice(["color", "size", "speed", "type"]),
            value=random.choice(["blue", "large", "fast", "complex"])
        )
        mem_type = MemoryType.SEMANTIC
    elif "Procedure step" in template:
        content = template.format(
            step=random.randint(1, 5),
            instruction=f"Execute {random.choice(actions)} operation"
        )
        mem_type = MemoryType.PROCEDURAL
    else:  # Emotional
        content = template.format(
            emotion=random.choice(emotions),
            subject=random.choice(subjects)
        )
        mem_type = MemoryType.EMOTIONAL

    memory_id = memory_system.store(
        content=content,
        memory_type=mem_type,
        priority=random.choice(list(MemoryPriority)),
        context={
            "test_batch": "performance",
            "index": i,
            "category": random.choice(["A", "B", "C"])
        },
        tags={f"tag_{random.randint(1, 5)}", "performance_test"},
        emotional_valence=random.uniform(-0.5, 0.5),
        importance_score=random.uniform(0.2, 0.9)
    )
    performance_test_ids.append(memory_id)

print(f"   ✅ Generated {len(performance_test_ids)} test memories")

# Wait for consolidation to process
time.sleep(2)

# Test search performance
print("   Testing search performance...")
search_start_time = time.time()

test_queries = [
    MemoryQuery(content_query="Alice", limit=10),
    MemoryQuery(memory_types=[MemoryType.EPISODIC], limit=20),
    MemoryQuery(context_filter={"category": "A"}, limit=15),
    MemoryQuery(content_query="pattern", memory_types=[MemoryType.SEMANTIC], limit=5)
]

total_results = 0
for query in test_queries:
    results = memory_system.search(query)
    total_results += len(results)

search_time = time.time() - search_start_time
print(f"   ✅ Search performance: {total_results} results in {search_time:.3f}s")

# Test retrieval performance
print("   Testing retrieval performance...")
retrieval_start_time = time.time()

# Randomly retrieve 20 memories
sample_ids = random.sample(performance_test_ids, min(20, len(performance_test_ids)))
retrieved_count = 0
for memory_id in sample_ids:
    memory = memory_system.retrieve(memory_id)
    if memory:
        retrieved_count += 1

retrieval_time = time.time() - retrieval_start_time
print(f"   ✅ Retrieval performance: {retrieved_count} memories in {retrieval_time:.3f}s")

# Memory system statistics after load
final_stats = memory_system.get_memory_statistics()
print(f"   Final system state: {final_stats['total_memories_created']} total memories")
print(f"   Consolidations: {final_stats['consolidation_count']}")
print()


📝 Step 6: Testing memory system performance...
   Generating memory load for performance testing...
   🏛️  Stored in long-term memory: 18ad7b96... (procedural)
   🏛️  Stored in long-term memory: 11b0a21b... (semantic)
   🏛️  Stored in long-term memory: 5c0bf421... (semantic)
   🏛️  Stored in long-term memory: a878e741... (semantic)
   🏛️  Stored in long-term memory: 1017669a... (semantic)
   🏛️  Stored in long-term memory: 9517955d... (semantic)
   🏛️  Stored in long-term memory: bd31b225... (semantic)
   🏛️  Stored in long-term memory: a9347216... (semantic)
   🏛️  Stored in long-term memory: fdf667fa... (semantic)
   🏛️  Stored in long-term memory: e85c0367... (episodic)
   🏛️  Stored in long-term memory: b064ebce... (semantic)
   🏛️  Stored in long-term memory: 1d328ba5... (emotional)
   🏛️  Stored in long-term memory: ad269d71... (semantic)
   🏛️  Stored in long-term memory: 5edc73df... (emotional)
   🏛️  Stored in long-term memory: bebbfec5... (semantic)
   🔄 Evicted 878b5036... f

In [11]:
# Step 7: Memory visualization and analysis
print("📝 Step 7: Creating memory visualization and analysis...")

visualizer = MemoryVisualizer(memory_system)

# Generate comprehensive dashboard
dashboard = visualizer.create_memory_dashboard()
print("   ✅ Generated memory dashboard")

# Analyze memory patterns
patterns = visualizer.analyze_memory_patterns(limit=50)
print(f"   ✅ Analyzed {patterns.get('sample_size', 0)} memories for patterns")

# Generate human-readable report
report = visualizer.generate_memory_report()
print("   ✅ Generated comprehensive memory report")

# Display key insights from dashboard
overview = dashboard['overview']
print(f"   Key insights:")
print(f"     Consolidation rate: {overview['consolidation_rate']:.1%}")
print(f"     System efficiency: {dashboard['working_memory_efficiency']['utilization']:.1f}%")

# Display memory distribution
distribution = dashboard['memory_distribution']['by_type_percentages']
print(f"     Memory type distribution:")
for mem_type, percentage in distribution.items():
    print(f"       {mem_type.capitalize()}: {percentage:.1f}%")

print()

📝 Step 7: Creating memory visualization and analysis...
📊 Memory visualizer initialized
   ✅ Generated memory dashboard
   ✅ Analyzed 48 memories for patterns
   ✅ Generated comprehensive memory report
   Key insights:
     Consolidation rate: 0.0%
     System efficiency: 100.0%
     Memory type distribution:
       Working: 0.0%
       Episodic: 7.3%
       Semantic: 58.5%
       Procedural: 17.1%
       Emotional: 17.1%



In [12]:
# Step 8: Advanced memory features demonstration
print("📝 Step 8: Demonstrating advanced memory features...")

# Test memory decay simulation
print("   Testing memory decay simulation...")
weak_memory_id = memory_system.store(
    content="This memory should decay quickly",
    memory_type=MemoryType.WORKING,
    priority=MemoryPriority.TRANSIENT,
    importance_score=0.1
)

# Time passage and access pattern
original_memory = memory_system.retrieve(weak_memory_id)
if original_memory:
    original_strength = original_memory.strength

    # Simulate decay
    original_memory.decay(decay_rate=0.1)
    print(f"     Memory strength: {original_strength:.2f} → {original_memory.strength:.2f}")

# Test emotional memory associations
print("   Testing emotional memory associations...")
emotional_memories = []

emotional_content = [
    ("Won the programming contest!", 0.8, 0.9),
    ("Had a difficult debugging session", -0.3, 0.7),
    ("Learned something new today", 0.5, 0.8),
    ("Code review went well", 0.4, 0.6)
]

for content, valence, importance in emotional_content:
    memory_id = memory_system.store(
        content=content,
        memory_type=MemoryType.EMOTIONAL,
        emotional_valence=valence,
        importance_score=importance,
        tags={"emotional", "experience"}
    )
    emotional_memories.append(memory_id)

# Create emotional associations
for i in range(len(emotional_memories) - 1):
    memory_system.associate(
        emotional_memories[i],
        emotional_memories[i + 1],
        "temporal_sequence",
        strength=0.6
    )

print(f"   ✅ Created {len(emotional_memories)} emotional memories with associations")

# Test memory consolidation monitoring
print("   Testing memory consolidation monitoring...")
consolidation_stats = memory_system.get_memory_statistics()
print(f"     Active consolidation queue: {consolidation_stats['consolidation_queue_size']} items")
print(f"     Total consolidations: {consolidation_stats['consolidation_count']}")

# Test agent learning and pattern recognition
print("   Testing advanced agent learning...")
learning_agent = MemoryAwareAgent(name="learning_agent", memory_system=memory_system)

# Feed structured learning data
learning_scenarios = [
    ("When I see error X, I should check Y", {"scenario": "debugging", "action": "check"}),
    ("User Alice prefers feature A", {"user": "Alice", "preference": "feature_A"}),
    ("Pattern B usually indicates problem C", {"pattern": "B", "indicates": "problem_C"}),
    ("When debugging error X, checking Y works", {"scenario": "debugging", "success": True}),
    ("Alice requested feature A again", {"user": "Alice", "repeated": True})
]

for scenario, context in learning_scenarios:
    learning_agent.process_with_memory(scenario, context)

# Check what the agent learned
agent_learning = learning_agent.get_learning_summary()
print(f"     Agent learned {agent_learning['meaningful_patterns']} patterns")
print(f"     Processing events: {agent_learning['learning_events']}")

if agent_learning['pattern_details']:
    print("     Top learned patterns:")
    for pattern, details in list(agent_learning['pattern_details'].items())[:3]:
        print(f"       {pattern}: {details['frequency']} occurrences")

print()

📝 Step 8: Demonstrating advanced memory features...
   Testing memory decay simulation...
   🔄 Evicted ecf21c25... from working memory
   💭 Stored in working memory: 922dd39f...
     Memory strength: 1.00 → 1.00
   Testing emotional memory associations...
   🏛️  Stored in long-term memory: 137a055d... (emotional)
   🔄 Evicted 58e9a7db... from working memory
   💭 Stored in working memory: 50258a5e...
   🏛️  Stored in long-term memory: ae4f5f18... (emotional)
   🔄 Evicted e69374c3... from working memory
   💭 Stored in working memory: 6a8e2bf1...
   ✅ Created 4 emotional memories with associations
   Testing memory consolidation monitoring...
     Active consolidation queue: 2 items
     Total consolidations: 0
   Testing advanced agent learning...
🧠 Created memory-aware agent: learning_agent
   🔄 Evicted 1875cc08... from working memory
   💭 Stored in working memory: 1213e95b...
   🔄 Evicted a3206938... from working memory
   💭 Stored in working memory: 7f8cf879...
   🔄 Evicted 9f3acf25..

In [13]:
# Step 9: Memory system integration testing
print("📝 Step 9: Testing memory system integration...")

# Test cross-system memory sharing
print("   Testing cross-system memory sharing...")

# Create multiple agents sharing the same memory system
agents = []
for i in range(3):
    agent = MemoryAwareAgent(
        name=f"shared_agent_{i}",
        memory_system=memory_system
    )
    agents.append(agent)

# Have agents interact and build shared memories
shared_interactions = [
    ("Agent 0 says: I found a bug in module A", {"speaker": "agent_0", "topic": "bug"}),
    ("Agent 1 says: I can help fix module A", {"speaker": "agent_1", "topic": "fix"}),
    ("Agent 2 says: Module A is now working", {"speaker": "agent_2", "topic": "success"})
]

for i, (interaction, context) in enumerate(shared_interactions):
    agent = agents[i % len(agents)]
    agent.process_with_memory(interaction, context)

print(f"   ✅ {len(agents)} agents shared {len(shared_interactions)} interactions")

# Test memory search across shared context
shared_query = MemoryQuery(
    content_query="module A",
    context_filter={"topic": "bug"},
    limit=5
)

shared_results = memory_system.search(shared_query)
print(f"   ✅ Found {len(shared_results)} shared memories about module A")

# Test system resilience
print("   Testing system resilience...")

# Simulate high load
rapid_storage_count = 0
start_time = time.time()

for i in range(20):
    memory_id = memory_system.store(
        content=f"Rapid test memory {i}",
        memory_type=random.choice(list(MemoryType)),
        priority=random.choice(list(MemoryPriority))
    )
    if memory_id:
        rapid_storage_count += 1

load_time = time.time() - start_time
print(f"   ✅ Stored {rapid_storage_count} memories under load in {load_time:.3f}s")

# Test error handling
print("   Testing error handling...")
try:
    # Try to retrieve non-existent memory
    fake_memory = memory_system.retrieve("fake_memory_id")
    print(f"     Error handling: {'✅ Graceful' if fake_memory is None else '❌ Failed'}")

    # Try to search with invalid query
    empty_results = memory_system.search(MemoryQuery(content_query=""))
    print(f"     Empty query handling: ✅ Returned {len(empty_results)} results")

except Exception as e:
    print(f"     ❌ Error handling failed: {e}")

print()

📝 Step 9: Testing memory system integration...
   Testing cross-system memory sharing...
🧠 Created memory-aware agent: shared_agent_0
🧠 Created memory-aware agent: shared_agent_1
🧠 Created memory-aware agent: shared_agent_2
   🔄 Evicted 4c0973f6... from working memory
   💭 Stored in working memory: 77bbf167...
   🔄 Evicted 03a24b2a... from working memory
   💭 Stored in working memory: b6e6e4eb...
   🔄 Evicted c3d78764... from working memory
   💭 Stored in working memory: edb0971c...
   🔄 Evicted 26122bff... from working memory
   💭 Stored in working memory: affe3365...
   🔄 Evicted b90a2cac... from working memory
   💭 Stored in working memory: 561622ab...
   🔄 Evicted 1b289ae5... from working memory
   💭 Stored in working memory: bc73428d...
   ✅ 3 agents shared 3 interactions
   ✅ Found 5 shared memories about module A
   Testing system resilience...
   🏛️  Stored in long-term memory: eac2afd3... (semantic)
   🏛️  Stored in long-term memory: 9b68f6cf... (episodic)
   🏛️  Stored in lon

In [14]:
# Step 10: Final analysis and cleanup
print("📝 Step 10: Final analysis and cleanup...")

# Generate final comprehensive report
print("   Generating final memory system report...")
final_report = visualizer.generate_memory_report()

# Display report summary
print("📋 MEMORY SYSTEM FINAL REPORT")
print("=" * 40)

# Extract key metrics from report
lines = final_report.split('\n')
for line in lines:
    if any(keyword in line.lower() for keyword in
           ['total memories', 'consolidation rate', 'utilization', 'most accessed']):
        print(f"   {line.strip()}")

print()

# Performance summary
final_stats = memory_system.get_memory_statistics()
print("📊 PERFORMANCE SUMMARY")
print("=" * 30)
print(f"   Total memories created: {final_stats['total_memories_created']}")
print(f"   Successful consolidations: {final_stats['consolidation_count']}")
print(f"   Total retrievals: {final_stats['retrieval_count']}")
print(f"   Working memory efficiency: {final_stats['working_memory']['utilization']:.1%}")
print(f"   Long-term memory size: {final_stats['long_term_memory']['total_memories']}")
print()

# Agent learning summary
print("🧠 AGENT LEARNING SUMMARY")
print("=" * 30)
for i, agent in enumerate(agents):
    learning = agent.get_learning_summary()
    print(f"   Agent {i}: {learning['meaningful_patterns']} patterns, {learning['learning_events']} events")

print()

# Knowledge graph summary
kg_final_stats = knowledge_graph.get_statistics()
print("🕸️  KNOWLEDGE GRAPH SUMMARY")
print("=" * 30)
print(f"   Entities: {kg_final_stats['total_entities']}")
print(f"   Relationships: {kg_final_stats['total_relationships']}")
print(f"   Entity types: {kg_final_stats['entity_types']}")
print()

# Cleanup
print("🧹 Cleaning up...")
memory_system.shutdown()
ltm.shutdown()

# Clean up database files
import os
for db_file in ["demo_memory.db", "integrated_memory.db"]:
    if os.path.exists(db_file):
        try:
            os.remove(db_file)
            print(f"   ✅ Cleaned up {db_file}")
        except:
            print(f"   ⚠️  Could not remove {db_file}")

print("   ✅ Memory system cleanup complete")
print()

print("✅ Tutorial 10 Complete!")
print()

📝 Step 10: Final analysis and cleanup...
   Generating final memory system report...
📋 MEMORY SYSTEM FINAL REPORT
   Total memories created: 106
   Consolidation rate: 0.0%
   Utilization: 100.0%
   Most accessed memory: 6 accesses

📊 PERFORMANCE SUMMARY
   Total memories created: 106
   Successful consolidations: 0
   Total retrievals: 25
   Working memory efficiency: 100.0%
   Long-term memory size: 56

🧠 AGENT LEARNING SUMMARY
   Agent 0: 0 patterns, 1 events
   Agent 1: 0 patterns, 1 events
   Agent 2: 0 patterns, 1 events

🕸️  KNOWLEDGE GRAPH SUMMARY
   Entities: 5
   Relationships: 5
   Entity types: 5

🧹 Cleaning up...
🛑 Memory system shutdown complete
   ✅ Cleaned up demo_memory.db
   ✅ Cleaned up integrated_memory.db
   ✅ Memory system cleanup complete

✅ Tutorial 10 Complete!



In [15]:
# SUMMARY OF WHAT WE LEARNED
# =============================================================================

print("📚 WHAT WE LEARNED:")

print("=" * 40)

print("1. 🧠 Built multi-layered memory architecture")

print("   - Working memory for active processing")

print("   - Long-term memory with different types (semantic, episodic, procedural, emotional)")

print("   - Memory consolidation and decay mechanisms")

print("   - Cross-memory querying and association systems")

print()

print("2. 🏗️  Implemented intelligent memory management")

print("   - Automatic routing between memory systems")

print("   - Background consolidation and maintenance")

print("   - Memory strength and importance modeling")

print("   - Persistent storage with SQLite")

print()

print("3. 🤖 Created memory-aware intelligent agents")

print("   - Context-driven memory retrieval")

print("   - Learning from experience and pattern recognition")

print("   - Conversation history and relationship tracking")

print("   - Shared memory systems for agent collaboration")

print()

print("4. 🕸️  Added knowledge graph capabilities")

print("   - Entity and relationship modeling")

print("   - Semantic memory organization")

print("   - Path finding and neighbor queries")

print("   - Structured knowledge representation")

print()

print("5. 📊 Built comprehensive analysis tools")

print("   - Memory system visualization and dashboards")

print("   - Performance monitoring and optimization")

print("   - Pattern analysis and reporting")

print("   - System health and efficiency metrics")

print()


📚 WHAT WE LEARNED:
1. 🧠 Built multi-layered memory architecture
   - Working memory for active processing
   - Long-term memory with different types (semantic, episodic, procedural, emotional)
   - Memory consolidation and decay mechanisms
   - Cross-memory querying and association systems

2. 🏗️  Implemented intelligent memory management
   - Automatic routing between memory systems
   - Background consolidation and maintenance
   - Memory strength and importance modeling
   - Persistent storage with SQLite

3. 🤖 Created memory-aware intelligent agents
   - Context-driven memory retrieval
   - Learning from experience and pattern recognition
   - Conversation history and relationship tracking
   - Shared memory systems for agent collaboration

4. 🕸️  Added knowledge graph capabilities
   - Entity and relationship modeling
   - Semantic memory organization
   - Path finding and neighbor queries
   - Structured knowledge representation

5. 📊 Built comprehensive analysis tools
   - Memor

In [16]:
# COMMON ERRORS AND SOLUTIONS
# =============================================================================

print("⚠️  COMMON ERRORS AND SOLUTIONS:")

print("=" * 40)

print("1. 🐛 Memory leaks and unbounded growth")

print("   Problem: Memory systems growing without bounds")

print("   Solution: Implement proper decay mechanisms and cleanup")

print("   Solution: Set limits on memory system sizes")

print("   Solution: Use background maintenance threads")

print()

print("2. 🐛 Slow memory retrieval performance")

print("   Problem: Linear search through large memory stores")

print("   Solution: Build proper indices (type, tag, context)")

print("   Solution: Use database storage with optimized queries")

print("   Solution: Cache frequently accessed memories")

print()

print("3. 🐛 Memory consolidation conflicts")

print("   Problem: Race conditions during consolidation")

print("   Solution: Use proper threading and synchronization")

print("   Solution: Implement consolidation queues and batching")

print("   Solution: Add transaction support for database operations")

print()

print("4. 🐛 Association explosion")

print("   Problem: Too many weak associations cluttering the system")

print("   Solution: Set minimum strength thresholds")

print("   Solution: Implement association decay and pruning")

print("   Solution: Limit association creation rules")

print()

print("5. 🐛 Context mismatch in retrieval")

print("   Problem: Retrieving irrelevant memories for current context")

print("   Solution: Improve context matching algorithms")

print("   Solution: Weight recent and relevant memories higher")

print("   Solution: Use semantic similarity for better matching")

print()

print("6. 🐛 Database corruption and persistence issues")

print("   Problem: SQLite database corruption or locking")

print("   Solution: Use proper connection management and timeouts")

print("   Solution: Implement backup and recovery mechanisms")

print("   Solution: Add data validation and integrity checks")

print()

print("7. 🐛 Memory type confusion")

print("   Problem: Wrong memory type assignment affecting retrieval")

print("   Solution: Clear guidelines for memory type classification")

print("   Solution: Automatic type inference based on content")

print("   Solution: Type validation and correction mechanisms")

print()

print("🎉 Ready for Tutorial 11: Deliberative Reasoning!")

print("   Next we'll explore advanced decision-making capabilities...")

print("\n🌟 You've now mastered:")

print("   • Individual agent intelligence and communication")

print("   • Multi-agent networks and collaborative processing")

print("   • Production monitoring, configuration, and management")

print("   • Structured agent circuits for complex workflows")

print("   • Sophisticated memory systems for context and learning")

print("\n🚀 Ready to build truly intelligent agent systems!")

⚠️  COMMON ERRORS AND SOLUTIONS:
1. 🐛 Memory leaks and unbounded growth
   Problem: Memory systems growing without bounds
   Solution: Implement proper decay mechanisms and cleanup
   Solution: Set limits on memory system sizes
   Solution: Use background maintenance threads

2. 🐛 Slow memory retrieval performance
   Problem: Linear search through large memory stores
   Solution: Build proper indices (type, tag, context)
   Solution: Use database storage with optimized queries
   Solution: Cache frequently accessed memories

3. 🐛 Memory consolidation conflicts
   Problem: Race conditions during consolidation
   Solution: Use proper threading and synchronization
   Solution: Implement consolidation queues and batching
   Solution: Add transaction support for database operations

4. 🐛 Association explosion
   Problem: Too many weak associations cluttering the system
   Solution: Set minimum strength thresholds
   Solution: Implement association decay and pruning
   Solution: Limit associ

---

🔒 **INTELLECTUAL PROPERTY & LICENSE NOTICE**

This tutorial and its contents — including code, architecture, narrative examples, and educational structure — are the intellectual property of **Shalini Ananda, PhD** and part of the **Neuron Framework** under a **Modified MIT License with Attribution**.

- Commercial use, redistribution, or derivative works **must** include clear and visible attribution to the original author.
- Use in products, consulting engagements, or educational materials **must reference this repository and author name.**
- Removal of author credit or misrepresentation of origin constitutes **a violation of the license and may trigger legal action.**
- You may **not white-label, obfuscate, or rebrand** this work without explicit, written permission.

Use of this tutorial in Colab or any other platform implies agreement with these terms.

📘 **License**: [LICENSE.md](../LICENSE.md)  
📌 **Notice**: [NOTICE.md](../NOTICE.md)  
🧠 **Author**: [Shalini Ananda, PhD](https://github.com/ShaliniAnandaPhD)